©2020-2021 ETH Zurich, Pagan Nicolò; D-ITET; Automatic Control Lab

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
import os
import sqlite3
from collections import Counter
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm    
from matplotlib.colors import LinearSegmentedColormap
import tikzplotlib as tikz
from scipy import interpolate, stats
import powerlaw

plt.rcParams["figure.figsize"]=[10, 7]

In [ ]:
# The following function finds the best linear log-log regression, and its goodness of fit parameters.
def linear_reg_log(x, y):
    print('Power law fit')
    x_log = np.log(x)
    y_log = np.log(y)
    pearson_coeff, p_value = stats.pearsonr(x_log, y_log)
    
    fit_line = np.poly1d(np.polyfit(x_log, y_log, deg=1))
    slope, intercept, r_value, p_value, std_err = stats.linregress(x_log, y_log)
    print ('slope:', slope, 'intercept:', intercept, 'r_value:', r_value, 'p_value:', p_value, 'std_err:', std_err)
    
    yhat = np.polyval(fit_line,x_log)
    SSE= sum((y_log-yhat)**2)
    MSE = SSE/len(y_log)
    RMSE = np.sqrt(MSE)
    print ('RMSE', RMSE)
    TSS = sum((y_log-np.mean(y_log))**2)     
    r_squared = 1 - (float(SSE))/TSS
    print ('r2', r_squared)
    adjusted_r_squared = 1 - (1-r_squared)*(len(y_log)-1)/(len(y_log)-1-1)
    print ('adjusted r2', adjusted_r_squared)
    y_fit = np.exp(yhat)
    return pearson_coeff, fit_line, y_fit


In [ ]:
# The function is used to make the colormap scale non-linear 
def scaled_cmap(original_map, scalingIntensity):
    R = []
    G = []
    B = [] 
    for i in range(255):
        R.append(original_map(i)[0])
        G.append(original_map(i)[1])
        B.append(original_map(i)[2])
    dataMax = 1;
    dataMin = 0;
    centerPoint = 0;
    x = np.linspace(1,255, 255)
    x = x - (centerPoint-dataMin)*len(x)/(dataMax-dataMin)
    x = scalingIntensity * x/np.max(np.abs(x))
    x = np.sign(x)* np.exp(np.abs(x))
    x = x - min(x); 
    x = x*511/max(x)+1
    fR = interpolate.interp1d(x, R)
    fG = interpolate.interp1d(x, G)
    fB = interpolate.interp1d(x, B)
    colormapR = fR(np.linspace(1,512,512))
    colormapG = fG(np.linspace(1,512,512))
    colormapB = fB(np.linspace(1,512,512))
    colormap = np.zeros([512,3])
    for i in range(512):
        colormap[i] = [colormapR[i], colormapG[i], colormapB[i]]
    colormap[-1] = [1, 1, 1]
    return LinearSegmentedColormap.from_list('mycmap', colormap, N=512)

In [ ]:
class Game:
    # To initialize the module, we need the name of the data-set, namely the name of the database, as well as the filtering criterion which consists of:
    # the dedication, i.e., the interest index, as defined in the main text
    # the final_date: we only consider nodes and edges created within the chosen date (01.10.2020)
    def __init__(self, name, folder_name, filtering):
        self.name = name
        self.folder_name = folder_name
        self.dedication = filtering['dedication']
        self.final_date = filtering['final_date']
        self.game_db = '{}.db'.format(name)
        self.n_nodes = 0
        self.n_edges = 0
        self.n_followers = 0
        if not os.path.exists(folder_name):
                os.makedirs(folder_name)

    
    # The following function gets the data from the database, which contains a user_db, and an edge_db
    # Entries can be removed, if the id is provided in the id_remove_list.
    def fetch_data(self, id_remove_list):
        conn = sqlite3.connect(self.game_db)
        self.nodes_df = pd.read_sql_query("SELECT id, created_at, dedication FROM user_db", conn)
        self.edges_df = pd.read_sql_query("SELECT source,target,date FROM edges_db", conn)
        self.nodes_df = self.nodes_df.loc[(self.nodes_df['dedication']>=self.dedication) & (pd.to_datetime(self.nodes_df['created_at']) <self.final_date) ]
        self.nodes_df = self.nodes_df[~self.nodes_df['Id'].isin(id_remove_list)]

        self.edges_df = self.edges_df.loc[(pd.to_datetime(self.edges_df['Date']) < self.final_date) & 
            (self.edges_df['Target'].isin(self.nodes_df.loc[:,'Id'].values))]
        self.edges_df.sort_values('Date', inplace = True)
        self.n_edges = self.edges_df.shape[0]
        self.outdegree_struct = Counter(elem for elem in tuple(self.edges_df['Source']))
        
        self.followers_df = self.edges_df['Source'].value_counts().rename_axis('Id').reset_index(name='Outdegree')
        self.n_followers = self.followers_df.shape[0]
        
        self.compute_indegree()
        self.n_nodes = len(self.nodes_df.index)
        
    # The nect function computes some basic statistics on the data-set    
    def print_statistics(self):
        print('Number of broadcasters:',  self.n_nodes)
        print('Number of edges:', self.n_edges)
        print('Number of followers:', self.n_followers)
        print('First edge: ' + str(pd.to_datetime(self.edges_df['Date'].head(1).item())))
        print('Last edge: '+ str(pd.to_datetime(self.edges_df['Date'].tail(1).item())))
     
    # The following function computes and stores the in-degree of the users in the database.
    def compute_indegree(self):
        indegree = self.edges_df['Target'].value_counts().rename_axis('Id').reset_index(name='Indegree')
        self.nodes_df = pd.merge(self.nodes_df, indegree, on='Id')
        self.nodes_df['Rank'] = self.nodes_df['Indegree'].rank(ascending=False)
        self.nodes_df.sort_values('Rank', inplace = True)    
    
    # The following function plots the in-degree as a function of the rank, for the first n_top_agents.
    # The data are fitted with a linear regression (after log-log conversion).
    def plot_Zipf(self, n_top_agents, save_plot):
        rank = self.nodes_df['Rank'][0:n_top_agents]
        indegree = self.nodes_df['Indegree'][0:n_top_agents]
        f_pearson_coeff, fit_line, y_fit = linear_reg_log(rank, indegree)  
        
        plt.figure()
        plt.plot(rank, indegree, 'o', markersize=2, label='origin')
        plt.plot(rank, y_fit,
               label="Slope={:.2f}, ".format(fit_line[1]) + 'Pearson coeff={:.2f}'.format(f_pearson_coeff))
        plt.yscale('log')
        plt.xscale('log')
        plt.legend(loc='upper right')
        plt.xlabel('Rank')
        plt.ylabel('Indegree')
        plt.title(self.name + ': top {} broadcasters'.format(n_top_agents))
        if (save_plot):
            tikz.save(self.folder_name+'Zipf.tikz', table_row_sep='\\\\\n')

       
    # In-degree analysis functions:
    
    # The following function plots the in-degree probability density function, and its power-law fit(s) and lognormal fit.
    def plot_indegree_pdf(self, save_plot):
        indegree = self.nodes_df['Indegree']
        xmax = max(indegree)
        xmin = min(indegree)
        
        n=len(indegree)
        zipfs_law_x = range(1,n)
        zipfs_law_y = xmax* np.power(np.linspace(1,n,n), -1)
        
        powerlawfit = powerlaw.Fit(indegree, discrete=True, estimate_discrete=True, fit_method='Likelihood')
        # As alternative, we also considere a power-law fit in which we define the xmin.
        powerlawfit_reduced = powerlaw.Fit(indegree, xmin=18000, discrete=True, estimate_discrete=True, fit_method='Likelihood') 
        xmin_PL = int(powerlawfit.power_law.xmin)
        xmin_PL_reduced = int(powerlawfit_reduced.power_law.xmin)
        powerlaw_fit_distribution = powerlaw.Power_Law(xmin=xmin_PL, parameters=[powerlawfit.alpha], discrete=True)  
        powerlaw_fit_reduced_distribution = powerlaw.Power_Law(xmin=xmin_PL_reduced, parameters=[powerlawfit_reduced.alpha], discrete=True)
        
        nbins = 50
        bins = np.logspace(np.log10(10), np.log10(xmax+1), nbins)
        widths = (bins[1:] - bins[:-1])
        hist = np.histogram(indegree, bins=bins)
        hist_norm = hist[0]/widths
        plt.scatter(bins[:-1], hist_norm, s=25, color='r', marker='o', label='Empirical data', alpha=0.5)
        plt.bar(bins[:-1], hist_norm, widths, bottom=10**(-6), alpha=0.5, label='Empirical histogram')
          
        hist_zipf = np.histogram(zipfs_law_y, bins=bins)
        hist_zipf_norm = hist_zipf[0]/widths
        plt.scatter(bins[:-1], hist_zipf_norm, s=25,  color='green', marker='o', label='Zipf\'s law', alpha=0.5)
        
        n_indegree = len(indegree)
        n_power_law = len(indegree[indegree>=xmin_PL])
        n_power_law_reduced = len(indegree[indegree>=xmin_PL_reduced])
        n_samples = 1000000
        
        powerlaw_random = powerlaw_fit_distribution.generate_random(n_samples)
        hist_powerlaw = np.histogram(powerlaw_random, bins=bins)
        hist_powerlaw_norm = hist_powerlaw[0]/widths/n_samples*n_power_law
        plt.loglog(bins[1:-1], hist_powerlaw_norm[1:], alpha=0.5,  label='fit pdf: alpha=' +str(powerlawfit.power_law.alpha)+' sigma='+str(powerlawfit.power_law.sigma)+' xmin='+str(int(powerlawfit.power_law.xmin)))
        
        powerlaw_reduced_random = powerlaw_fit_reduced_distribution.generate_random(n_samples)
        hist_powerlaw_reduced = np.histogram(powerlaw_reduced_random, bins=bins)
        hist_powerlaw_reduced_norm = hist_powerlaw_reduced[0]/widths/n_samples*n_power_law_reduced
        plt.loglog(bins[1:-1], hist_powerlaw_reduced_norm[1:], alpha=0.5,  label='fit pdf: alpha=' +str(powerlawfit_reduced.power_law.alpha)+' sigma='+str(powerlawfit_reduced.power_law.sigma)+' xmin='+str(int(powerlawfit_reduced.power_law.xmin)))
        
        
        sigma, loc, scale = stats.lognorm.fit(indegree, floc=0)
        logn_samples = np.random.lognormal(np.log(scale), sigma, n_samples)
        hist_logn = np.histogram(logn_samples, bins=bins)
        hist_logn_norm = hist_logn[0]/widths/len(logn_samples)*n_indegree
        plt.loglog(bins[:-1], hist_logn_norm, alpha=0.5,  label='lognormal fit mu='+str(np.log(scale))+' sigma='+str(sigma))
        plt.title('Indegree Probability Density Function and power-law fit, normalized after xmin')
        plt.legend(loc="upper right")
        plt.xlabel('indegree')
        plt.ylabel('pdf')
        if (save_plot):
             tikz.save(self.folder_name+'indegree_pdf.tikz', table_row_sep='\\\\\n')
        plt.show()

        
    # The following function plots the in-degree complementary cumulative distribution function, and its power-law fit(s) and lognormal fit.    
    def plot_indegree_ccdf(self, save_plot):
        indegree = self.nodes_df['Indegree']
        xmax = max(indegree)
        xmin = min(indegree)
        
        n=len(indegree)
        zipfs_law_x = range(1,n)
        zipfs_law_y = xmax* np.power(np.linspace(1,n,n), -1)
        
        
        powerlawfit = powerlaw.Fit(indegree, discrete=True, estimate_discrete=True, fit_method='Likelihood')
        # As alternative, we also considere a power-law fit in which we define the xmin.
        powerlawfit_reduced = powerlaw.Fit(indegree, xmin=18000, discrete=True, estimate_discrete=True, fit_method='Likelihood') 
        xmin_PL = int(powerlawfit.power_law.xmin)
        xmin_PL_reduced = int(powerlawfit_reduced.power_law.xmin)
        powerlaw_fit_distribution = powerlaw.Power_Law(xmin=xmin_PL, parameters=[powerlawfit.alpha], discrete=True) # xmin=2 might not be good for different values of N. This is a bit of a hack, because of the powerlaw interpolation package 
        powerlaw_fit_reduced_distribution = powerlaw.Power_Law(xmin=xmin_PL_reduced, parameters=[powerlawfit_reduced.alpha], discrete=True) # xmin=2 might not be good for different values of N. This is a bit of a hack, because of the powerlaw interpolation package 
     
        nbins = 100
        bins = np.logspace(np.log10(xmin), np.log10(xmax+1), nbins)
        widths = (bins[1:] - bins[:-1])
        hist = np.histogram(indegree, bins=bins)
        hist_norm = hist[0]/sum(hist[0])
        ccdf = np.cumsum(hist_norm[::-1])[::-1] 
        plt.plot(bins[:-1], ccdf, color='r', marker='o', label='Empirical data', alpha=0.5)
        plt.bar(bins[:-1], ccdf, widths, bottom=10**(-4), alpha=0.5, label='Empirical histogram')
        
        
        hist_zipf = np.histogram(zipfs_law_y, bins=bins)
        hist_zipf_norm = hist_zipf[0]/sum(hist_zipf[0])
        ccdf_zipf = np.cumsum(hist_zipf_norm[::-1])[::-1] 
        plt.plot(bins[:-1], ccdf_zipf, color='green', marker='o', label='Zipf\'s law', alpha=0.5)
          
        n_indegree = len(indegree)
        n_power_law = len(indegree[indegree>=xmin_PL])
        n_power_law_reduced = len(indegree[indegree>=xmin_PL_reduced])
        n_samples = 1000000
    
        plt.loglog(np.linspace(xmin_PL,xmax, 200), powerlaw_fit_distribution.ccdf(np.linspace(xmin_PL,xmax, 200))*n_power_law/n_indegree, label='fit ccdf: alpha=' +str(powerlawfit.power_law.alpha)+' sigma='+str(powerlawfit.power_law.sigma)+' xmin='+str(int(powerlawfit.power_law.xmin)), alpha=0.5)
        plt.loglog(np.linspace(xmin_PL_reduced,xmax, 200), powerlaw_fit_reduced_distribution.ccdf(np.linspace(xmin_PL_reduced,xmax, 200))*n_power_law_reduced/n_indegree, label='fit ccdf: alpha=' +str(powerlawfit_reduced.power_law.alpha)+' sigma='+str(powerlawfit_reduced.power_law.sigma)+' xmin='+str(int(powerlawfit_reduced.power_law.xmin)), alpha=0.5)
    
        sigma, loc, scale = stats.lognorm.fit(indegree, floc=0)
        logn_samples = np.random.lognormal(np.log(scale), sigma, n_samples)
        hist_logn = np.histogram(logn_samples, bins=bins)
        hist_logn_norm = hist_logn[0]/sum(hist_logn[0])
        logn_ccdf = np.cumsum(hist_logn_norm[::-1])[::-1]
        plt.loglog(bins[:-1], logn_ccdf, alpha=0.5, label='lognormal fit mu='+str(np.log(scale))+' sigma='+str(sigma))
        
        plt.title('Indegree Complementary Cumulative Distribution Function and power-law fit, normalized after xmin')
        plt.legend(loc="upper right")
        plt.xlabel('indegree')
        plt.ylabel('ccdf')
        if (save_plot):
            tikz.save(self.folder_name+'indegree_ccdf.tikz', table_row_sep='\\\\\n')
        plt.show()
    
    # Out-degree analysis functions:
    
     # The following function plots the outdegree distribution, as reported in Fig. 8
    def plot_outdegree_distribution(self, save_plot):
        outdegree_df = self.followers_df['Outdegree'].value_counts().rename_axis('Outdegree').reset_index(name='Count')
        outdegree_df = outdegree_df.append({'Outdegree': 0, 'Count': 0}, ignore_index=True)
        outdegree_df.sort_values('Outdegree', inplace=True, ascending=True)
     
        outdegree_df['Count'] = outdegree_df['Count'].divide(sum(outdegree_df['Count']))
        outdegree_df['Sum_count'] = 1-np.cumsum(outdegree_df['Count'])
        outdegree_df['Outdegreeplusone'] = outdegree_df['Outdegree']+1
        
        fig, ax = plt.subplots()
        ax = outdegree_df.plot(x= 'Outdegreeplusone', y ='Sum_count', kind='line', label='Data', ax=ax)
        plt.legend(loc='upper right')
        plt.xlim([1, 150])
        plt.xscale('log')
        plt.yscale('log')
        plt.ylim([10**-7, 1])
        plt.xlabel('Outdegree')
        plt.ylabel('Frequency')
        plt.title('{} Nodes outdegree distribution'.format(self.name))
        if (save_plot):
            tikz.save(self.folder_name+'outdegree_ccdf.tikz', table_row_sep='\\\\\n')
    
    # The next function computes the requested percentile of the empirical out-degree distribution
    def compute_outdegree_percentile(self, percentile):
        return np.percentile(game.followers_df['Outdegree'].tolist(), percentile)
            
    # The next function computes the out-degree distribution of the followers of the "n_top_agents" users.
    def compute_followers_outdegree_distribution(self, n_top_agents):
        broadcasters_id = tuple(self.nodes_df[0:n_top_agents]['Id']) 
        followers_outdegree_distribution = [[0] * (max(self.followers_df['Outdegree'])+1) for _ in range(len(broadcasters_id))]
        
        for broadcaster, f in zip(broadcasters_id, followers_outdegree_distribution):
            followers_list = self.edges_df.loc[self.edges_df['Target'] == broadcaster, 'Source'].tolist()
            for follower in followers_list:
                outdegree = self.outdegree_struct.get(follower)
                f[outdegree-1] = f[outdegree-1]+1 
                #storing the count for outdegree i into the cell i-1, given that for i=0 the count is 0.
        return followers_outdegree_distribution
        
    # The next function provides the plot of the out-degree distribution, aggregated per each of the first n_top_agents broadcasters
    def plot_outdegree_distribution_stacked(self, n_top_agents, save_plot):    
        followers_outdegree_distribution = self.compute_followers_outdegree_distribution(n_top_agents)
        # We only consider those followers with outdegree of at most 20 (which coincides with more than 99% of the followers)
        for ele in followers_outdegree_distribution:
            del ele[20:]
              
        followers_outdegree_distribution_normalized = [[j / sum(i) if sum(i) != 0 else 0 for j in i] for i in followers_outdegree_distribution]
    
        df = pd.DataFrame(followers_outdegree_distribution, index=list(range(1, n_top_agents + 1)))
        df_norm = pd.DataFrame(followers_outdegree_distribution_normalized, index=list(range(1, n_top_agents + 1)))

        plt.figure(0)
        df_norm.plot(kind='bar', stacked='True', legend=None)
        plt.xlabel('Rank')
        plt.ylabel('Frequency')
        plt.title(self.name + ':#Followers')
        plt.legend(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'], title='Outdegree')
        if (save_plot):
            tikz.save(self.folder_name+'followers_outdegree_dist_normalized.tikz', table_row_sep='\\\\\n')

                
    # Overlap analysis functions:
    
    # The next function computes the overlap matrix, as defined in the manuscript
    def create_heatmap(self, n_top_agents):
        heatmap_new = np.zeros([n_top_agents+1,n_top_agents+1])
        broadcasters_id = tuple(self.nodes_df[0:n_top_agents+1]['Id'])
        n_i =0
        for i in broadcasters_id:
            n_j = 0
            row_i = []
            i_followers = self.edges_df.loc[self.edges_df['Target'] == i, 'Source'].tolist()
            i_followers = set(i_followers)
            for j in broadcasters_id:
                j_followers = self.edges_df.loc[self.edges_df['Target'] == j, 'Source'].tolist()
                j_followers = set(j_followers)
                row_i.append(len(j_followers & i_followers) / len(i_followers))
                heatmap_new[n_i, n_j] = len(j_followers & i_followers) / len(i_followers)
                n_j = n_j +1
            n_i = n_i+1
        return heatmap_new
    
    # The next function plots (and saves) the overlap matrix, as defined in the manuscript.
    def plot_heatmap(self, size, save_plot):
        heatmap = self.create_heatmap(size)
        y, x = np.mgrid[slice(0, size+1, 1), slice(0, size+1, 1)]
        plt.pcolor(x, y, heatmap[0:size+1, 0:size+1], cmap=scaled_cmap(cm.hot, 2))
        plt.xlim(0,size)
        plt.ylim(0,size)
        plt.gca().invert_yaxis()
        plt.colorbar()
        if (save_plot):
            dict = {'x': (np.reshape(x+0.5, ((size+1)**2),1)), 'y': (np.reshape(y+0.5, ((size+1)**2),1)), 'c': (np.reshape(heatmap, ((size+1)**2),1))}
            df = pd.DataFrame(dict)
            df.to_csv(self.folder_name+'overlap_matrix.csv', index=False, sep ='\t')
        plt.xlabel('Rank j')
        plt.ylabel('Rank i')
        plt.show()
        

In [ ]:
# For the analysis of the Chess data-set, uncomment the following three lines
# name = 'Chess' # --> the name of the database (without .db) NOTE: The database must be saved in the same folder
# folder_name = 'Chess_Results/'
# id_remove_list = [158881542]

# For the analysis of the Poker data-set, uncomment the following three lines
# name = 'Poker' # --> the name of the database (without .db) NOTE: The database must be saved in the same folder
# folder_name = 'Poker_Results/'
# id_remove_list = [55481471] 

# Use the following line to change the filtering criterion on the interest-index (or dedication)
filtering = {'dedication': 0.8, 'final_date' :'2020-10-01'}
game = Game(name, folder_name, filtering)

# For each of the chess and poker data-sets, two user-ids were removed after manual inspection of the content of the channels.
# Apparently, even though these users were recently streaming in these categories, much of their followers had very poor overlap with the followers of others,
# which indicates that their followers are due to a different (possibly older) interest.
game.fetch_data(id_remove_list)

First, we print some basics statistics related to the data-set.

In [ ]:
game.print_statistics()

# Zipf's law
The following code provides the results reported in Fig. 4 in the manuscript.

In [ ]:
n_top_agents = 15
save_plot = True
game.plot_Zipf(n_top_agents, save_plot)

# In-degree distribution
In the following, we provide the code for the in-degree distribution analysis shown in Supplementary Fig. 12. First, we show the probability density function, and the various fits.

In [ ]:
save_plot = True
game.plot_indegree_pdf(save_plot)

Then, we show the complementary cumulative distribution function.

In [ ]:
save_plot = True
game.plot_indegree_ccdf(save_plot)

# Out-degree distribution
The following code provides the results reported in Fig. 8 in the manuscript.

In [ ]:
save_plot = True
game.plot_outdegree_distribution(save_plot)

We also compute the 99th percentile of the distribution, as reported in the manuscript

In [ ]:
game.compute_outdegree_percentile(99)

Finally, we provide the plot of the out-degree distribution, aggregated per each followers. The number of followers is normalized with the total number of followers of the broadcaster. The data are reported in Supplementary Fig. 15.

In [ ]:
n_top_agents = 15
save_plot = True
game.plot_outdegree_distribution_stacked(n_top_agents, save_plot)

# Overlap analysis
The following code provides the results reported in Fig. 10b, and in Supplementary Figs. 10, 11, 14

In [ ]:
save_plot = True
n_top_agents = 15
game.plot_heatmap(n_top_agents, save_plot)